# <center><b><font color="red">PyTorch Going Modular</font></b></center>

## **Script Mode**

In [ ]:
# Create a Python file called hello_world.py
%%writefile hello_world.py
print("I Love PyTorch")

Writing hello_world.py


In [ ]:
# Run this Python file
!python hello_world.py

I Love PyTorch


In [ ]:
! rm -rf hello_world.py

## **1. Get data**

In [ ]:
import requests
import zipfile
from pathlib import Path

# Step 1: Set up the path
url = "https://github.com/mohd-faizy/PyTorch-Essentials/raw/main/_datasets/pizza_steak_sushi.zip"
image_path = Path("data/pizza_steak_sushi")
image_path.mkdir(parents=True, exist_ok=True)
zip_path = image_path.parent / "pizza_steak_sushi.zip"

# Step 2: Download the zip file containing the data
zip_path.write_bytes(requests.get(url).content)

# Step 3: Unzip the downloaded file into the image directory
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(image_path)

## **2. Create Datasets and DataLoaders**

## **3. Making a model (TinyVGG)**




## **4. Creating `train_step()` and `test_step()` functions and `train()` to combine them.**

## **5. Creating a function to save the model**

## **6. Train, evaluate and save the model**